In [ ]:
import matplotlib, sys,  os
import numpy as np
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from tqdm import tqdm

import healpy as hp
import sys
import os
sys.path.append("/home/lhaaso/caowy/Public/3MLWCDA/Standard/src")
import mylib as my
import ROOT as rt
import root_numpy as rn
import uproot

In [ ]:
savename="WCDA"
nside=1024
npix=hp.nside2npix(nside)
skymapson=np.array([np.zeros(npix) for _ in range(14)])
skymapsoff=np.array([np.zeros(npix) for _ in range(14)])
alpha=np.array([np.zeros(npix) for _ in range(14)])
for i in tqdm(range(1024)):
    try:
        data=np.loadtxt(f"{savename}_sky_on_off_part{i}.txt")
        data = data.T
    except:
        continue
    if data.size==0:
        continue
    for j in range(6):
        dataindexon = data[2][(data[1].astype(int)==0) & (data[0].astype(int)==j)].astype(int)
        dataon = data[3][(data[1].astype(int)==0) & (data[0].astype(int)==j)].astype(int)
        dataindexoff = data[2][(data[1].astype(int)==1) & (data[0].astype(int)==j)].astype(int)
        dataoff = data[3][(data[1].astype(int)==1) & (data[0].astype(int)==j)]
        dataindexoffs = data[2][(data[1].astype(int)==2) & (data[0].astype(int)==j)].astype(int)
        dataoffs = data[3][(data[1].astype(int)==2) & (data[0].astype(int)==j)]
        skymapson[j][dataindexon]=dataon.astype(int)
        skymapsoff[j][dataindexoff]=dataoff
        alpha[j][dataindexoffs]=dataoffs

In [ ]:
%matplotlib inline
pixIdx = np.arange(npix) # pixel index numbers
new_lats = hp.pix2ang(nside, pixIdx)[0] # thetas I need to populate with interpolated theta values
new_lons = hp.pix2ang(nside, pixIdx)[1] # phis, sameA
mask = ((-new_lats + np.pi/2 < -25./180*np.pi) | (-new_lats + np.pi/2 > 85./180*np.pi)  ) 
for j in tqdm(range(6)):
    aa = alpha[j]
    aa[mask]=hp.UNSEEN
    aa = hp.ma(aa)
    
    # my.hpDraw("region_name", "Modelname", on,0,0,skyrange=(0,360,-20,80),
    #             colorlabel="Significance", contours=[1000], save=False, cat={}, color="Milagro",  xsize=2048, plotmol=1, savename=savename)
    # plt.savefig(f"1fullsky_{savename}_on_{j}.pdf",dpi=300)

    # plt.figure()
    # my.hpDraw("region_name", "Modelname", off,0,0,skyrange=(0,360,-20,80),
    #             colorlabel="Significance", contours=[1000], save=False, cat={}, color="Milagro", xsize=2048, plotmol=1, savename=savename)
    # plt.savefig(f"1fullsky_{savename}_off_{j}.pdf",dpi=300)

    plt.figure()
    my.hpDraw("region_name", "Modelname", aa,0,0,skyrange=(0,360,-20,80),
                colorlabel="Significance", contours=[1000], save=False, cat={}, color="Milagro", xsize=2048, plotmol=1, savename=savename)
    plt.savefig(f"1fullsky_{savename}_alpha_{j}.pdf",dpi=300)

    # nestid = hp.ring2nest(1024, np.arange(hp.nside2npix(1024)))
    # nestmap=np.zeros(hp.nside2npix(1024))
    # nestmap[nestid] = skymap
    # nestmap = hp.ma(nestmap)
    # plt.figure()
    # hp.mollview(on)
    # hp.write_map(f"1fullsky_{savename}_on_{j}.fits.gz",on,overwrite=True)

    # plt.figure()
    # hp.mollview(off)
    # hp.write_map(f"1fullsky_{savename}_off_{j}.fits.gz",off,overwrite=True)
    
    plt.figure()
    hp.mollview(aa)
    hp.write_map(f"1fullsky_{savename}_alpha_{j}.fits.gz",aa,overwrite=True)
    # map2 = my.change_coord(skymap,["C","G"])
    # hp.write_map(f"fullsky_{savename}.fits",map2,overwrite=True)

In [ ]:
fout = rt.TFile.Open(f"./test.root", 'recreate')
tree = rt.TTree("BinInfo", "BinInfo")
name = bytearray(1)
duration = np.array([0], dtype=np.float64)
totalDuration = np.array([0], dtype=np.float64)
startMJD = np.array([0], dtype=np.float64)
stopMJD = np.array([0], dtype=np.float64)

# 将变量与branch关联
tree.Branch("name", name, "name/s")
tree.Branch("duration", duration, "duration/D")
tree.Branch("totalDuration", totalDuration, "totalDuration/D")
tree.Branch("startMJD", startMJD, "startMJD/D")
tree.Branch("stopMJD", stopMJD, "stopMJD/D")

# 填充数据
nside=1024
npix=hp.nside2npix(nside)
dtype1 = [('count', float)]
dtype2 = [('count', float)]
skymaponout=np.zeros(npix, dtype = dtype1)
skymapoffout=np.zeros(npix, dtype = dtype2)
pixid = np.arange(npix)
pixarea= 4*np.pi/npix

data_name = "/home/lhaaso/caowy/data.root"
file = uproot.open(data_name)
histogram_on = file["all_sky_cube_on;1"]
histogram_off = file["all_sky_cube_bg;1"]
header = file["header"].title.split(",")
header_value=file["header"].values()
print(header,header_value)

for i in range(14):
    name[0]=i
    duration[0] = 10
    totalDuration[0] = header_value[1]*24 #*0.99726956
    startMJD[0]=58844.0
    stopMJD[0]=60156.0
    tree.Fill()
fout.Write()
fout.Close()

for i in tqdm(range(14)):
    print('nHit%02d'%int(i))
    fout = rt.TFile.Open(f"./test.root", 'UPDATE')
    ndir=fout.mkdir('nHit%02d'%int(i),'nHit%02d'%int(i))

    tree1=rt.TTree('data','data')
    tree2=rt.TTree('bkg','bkg')
    obj1 = rt.TParameter(int)("Nside",1024)
    obj2 = rt.TParameter(int)("Scheme",0)

    tree1.SetDirectory(ndir)
    tree1.SetEntries(npix)
    tree2.SetDirectory(ndir)
    tree2.SetEntries(npix)

    tree1.GetUserInfo().Add(obj1)
    tree1.GetUserInfo().Add(obj2)
    tree2.GetUserInfo().Add(obj1)
    tree2.GetUserInfo().Add(obj2)

    skymaponout = hp.read_map(f"./1fullsky_KM2A_on_{i}.fits.gz")
    skymapoffout = hp.read_map(f"./1fullsky_KM2A_off_{i}.fits.gz")
    
    rn.array2tree(skymaponout.astype([('count', float)]),tree=tree1)
    rn.array2tree(skymapoffout.astype([('count', float)]),tree=tree2)
    fout.Write()
    fout.Close()

In [ ]:
file = uproot.open("/home/lhaaso/caowy/20210305_20230731_bkgJ2000.root")
header = file["Map_header"].arrays(library="np")

In [ ]:
header

In [ ]:
fout = rt.TFile.Open(f"./test2.root", 'recreate')
tree = rt.TTree("BinInfo", "BinInfo")
name = bytearray(1)
duration = np.array([0], dtype=np.float64)
totalDuration = np.array([0], dtype=np.float64)
startMJD = np.array([0], dtype=np.float64)
stopMJD = np.array([0], dtype=np.float64)

# 将变量与branch关联
tree.Branch("name", name, "name/s")
tree.Branch("duration", duration, "duration/D")
tree.Branch("totalDuration", totalDuration, "totalDuration/D")
tree.Branch("startMJD", startMJD, "startMJD/D")
tree.Branch("stopMJD", stopMJD, "stopMJD/D")

# 填充数据
nside=1024
npix=hp.nside2npix(nside)
dtype1 = [('count', float)]
dtype2 = [('count', float)]
skymaponout=np.zeros(npix, dtype = dtype1)
skymapoffout=np.zeros(npix, dtype = dtype2)
pixid = np.arange(npix)
pixarea= 4*np.pi/npix

file = uproot.open("/home/lhaaso/caowy/20210305_20230731_bkgJ2000.root")
header = file["Map_header"].arrays(library="np")

for i in range(6):
    name[0]=i
    duration[0] = 10
    totalDuration[0] = 68771059/60/60
    startMJD[0]=59277.67261921
    stopMJD[0]=60156.66609676
    tree.Fill()
fout.Write()
fout.Close()

for i in tqdm(range(6)):
    print('nHit%02d'%int(i))
    fout = rt.TFile.Open(f"./test2.root", 'UPDATE')
    ndir=fout.mkdir('nHit%02d'%int(i),'nHit%02d'%int(i))

    tree1=rt.TTree('data','data')
    tree2=rt.TTree('bkg','bkg')
    obj1 = rt.TParameter(int)("Nside",1024)
    obj2 = rt.TParameter(int)("Scheme",0)

    tree1.SetDirectory(ndir)
    tree1.SetEntries(npix)
    tree2.SetDirectory(ndir)
    tree2.SetEntries(npix)

    tree1.GetUserInfo().Add(obj1)
    tree1.GetUserInfo().Add(obj2)
    tree2.GetUserInfo().Add(obj1)
    tree2.GetUserInfo().Add(obj2)

    skymaponout = hp.read_map(f"./1fullsky_WCDA_on_{i}.fits.gz")
    skymapoffout = hp.read_map(f"./1fullsky_WCDA_off_{i}.fits.gz")
    
    rn.array2tree(skymaponout.astype([('count', float)]),tree=tree1)
    rn.array2tree(skymapoffout.astype([('count', float)]),tree=tree2)
    fout.Write()
    fout.Close()

In [ ]:
skymaponout.astype([('count', float)])